In [ ]:
############## URUCHOM TO NAJPIERW! ###################

# zaimportuj wszystko i zdefiniuj funkcję wykonującą testy
from importlib import reload
from helper import run
import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [ ]:
from helper import hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
h = hash256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

In [ ]:
from helper import hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
for item in (b'hello world', b'goodbye'):
    h = hash256(item)
    bit = int.from_bytes(h, 'big') % bit_field_size
    bit_field[bit] = 1
print(bit_field)

### Ćwiczenie 1

Oblicz wynik filtra Blooma dla sekwencji `hello world` i `goodbye`, używając funkcji skrótu hash160 dla pola bitowego o długości 10.


In [ ]:
# Ćwiczenie 1

from helper import hash160

bit_field_size = 10
bit_field = [0] * bit_field_size
items = (b'hello world', b'goodbye')
# wykonaj w pętli dla każdego elementu
    # oblicz hash160 elementu
    # zinterpretuj skrót, jako liczbę całkowitą big endian i podziel modulo przez nią bit_field_size
    # ustaw ten bit w polu bit_field
# wydrukuj bit_field

In [ ]:
from helper import hash256, hash160
bit_field_size = 10
bit_field = [0] * bit_field_size
for item in (b'hello world', b'goodbye'):
    for hash_function in (hash256, hash160):
        h = hash_function(item)
        bit = int.from_bytes(h, 'big') % bit_field_size
        bit_field[bit] = 1
print(bit_field)

In [ ]:
from helper import murmur3
from bloomfilter import BIP37_CONSTANT
field_size = 2
num_functions = 2
tweak = 42
bit_field_size = field_size * 8
bit_field = [0] * bit_field_size
for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

### Ćwiczenie 2

Dla filtra Blooma o następujących parametrach: `size=10`, `function_count=5`, `tweak=99` określ, które bity zostaną ustawione po dodaniu tych elementów. (Użyj funkcji `bit_field_to_bytes` do konwersji na bajty).

* `b'Hello World'`
* `b'Goodbye!'`

In [ ]:
# Ćwiczenie 2

from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import bit_field_to_bytes, murmur3

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')
# oblicz rozmiar pola bitowego
# utwórz puste pole bitowe
# wykonaj w pętli dla każdego elementu
    # wykonaj w pętli dla wszystkich funkcji
        # oblicz ziarno
        # oblicz skrót murmur3 elementu używając ziarna
        # podziel modulo przez rozmiar pola bitowego
        # ustaw bit
# skonwertuj pole bitowe na bajty
# wydrukuj bajty szesnastkowo

### Ćwiczenie 3

Napisz metodę `add` dla klasy `BloomFilter`.

#### Wykonaj [ten test](/edit/r12/bloomfilter.py): `bloomfilter.py:BloomFilterTest:test_add`

In [ ]:
# Ćwiczenie 3

reload(bloomfilter)
run(bloomfilter.BloomFilterTest("test_add"))

### Ćwiczenie 4

Napisz metodę `filterload` dla klasy `BloomFilter`.

#### Wykonaj [ten test](/edit/r12/bloomfilter.py): `bloomfilter.py:BloomFilterTest:test_filterload`

In [ ]:
# Ćwiczenie 4

reload(bloomfilter)
run(bloomfilter.BloomFilterTest("test_filterload"))

### Ćwiczenie 5

Napisz metodę `serialize` dla klasy `GetDataMessage`.

#### Wykonaj [ten test](/edit/r12/network.py): `network.py:GetDataMessageTest:test_serialize`

In [ ]:
# Ćwiczenie 5

reload(network)
run(network.GetDataMessageTest("test_serialize"))

In [ ]:
from bloomfilter import BloomFilter
from helper import decode_base58
from merkleblock import MerkleBlock
from network import FILTERED_BLOCK_DATA_TYPE, GetHeadersMessage, GetDataMessage, HeadersMessage, SimpleNode
from tx import Tx
last_block_hex = '00000000000538d5c2246336644f9a4956551afb44ba47278759ec55ea912e19'
address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
h160 = decode_base58(address)
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True, logging=False)
bf = BloomFilter(size=30, function_count=5, tweak=90210)
bf.add(h160)
node.handshake()
node.send(bf.filterload())
start_block = bytes.fromhex(last_block_hex)
getheaders = GetHeadersMessage(start_block=start_block)
node.send(getheaders)
headers = node.wait_for(HeadersMessage)
getdata = GetDataMessage()
for b in headers.blocks:
    if not b.check_pow():
        raise RuntimeError('proof of work is invalid')
    getdata.add_data(FILTERED_BLOCK_DATA_TYPE, b.hash())
node.send(getdata)
found = False
while not found:
    message = node.wait_for(MerkleBlock, Tx)
    if message.command == b'merkleblock':
        if not message.is_valid():
            raise RuntimeError('invalid merkle proof')
    else:
        for i, tx_out in enumerate(message.tx_outs):
            if tx_out.script_pubkey.address(testnet=True) == address:
                print('found: {}:{}'.format(message.id(), i))
                found = True
                break

### Ćwiczenie 6

Pobierz identyfikator bieżącego bloku w testnecie prześlij sobie kilka tBTC, znajdź UTXO odpowiadający tym tBTC bez użycia eksploratora bloków, utwórz transakcję, wykorzystując ten UTXO jako wejście, i wyślij komunikat `tx` w sieci testnet.

In [ ]:
# Ćwiczenie 6

import time

from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import hash256, little_endian_to_int, encode_varint, read_varint, decode_base58, SIGHASH_ALL
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    NetworkEnvelope,
    SimpleNode,
    TX_DATA_TYPE,
    FILTERED_BLOCK_DATA_TYPE,
)
from script import p2pkh_script, Script
from tx import Tx, TxIn, TxOut

last_block_hex = ''  # FILL THIS IN

secret = little_endian_to_int(hash256(b''))  # FILL THIS IN
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
h160 = decode_base58(addr)

target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 5000  # opłata w satoshi


# połącz się z testnet.programmingbitcoin.com w trybie testnetu
# Utwórz filtr Blooma o rozmiarze 30, z 5 funkcjami. Dodaj tweak.
# dodaj h160 do filtra Blooma
# zakończ uzgadnianie
# załaduj filtr Blooma za pomocą komunikatu filterload

# ustaw blok startowy na last_block otrzymaną powyżej
# wyślij komunikat getheaders z blokiem początkowym

# zaczekaj na komunikat headers
# zapisz ostatni blok, jako None
# zainicjalizuj GetDataMessage
# przejdź pętlą przez bloki w nagłówkach
    # sprawdź czy dowód pracy w bloku jest prawidłowy
    # sprawdź czy prev_block tego bloku jest ostatnim blokiem
    # dodaj nowy element do komunikatu getdata
    # powinno być FILTERED_BLOCK_DATA_TYPE i skrót bloku
    # ustaw ostatni blok na bieżący skrót
# wyślij komunikat getdata

# zainicjuj prev_tx i prev_index na None
# wykonuj w pętli gdy prev_tx ma wartość None
    # czekaj na komunikat merkleblock lub tx
    # jeśli mamy komunikat merkleblock
        # sprawdź, czy MerkleBlock jest poprawny
    # w przeciwnym wypadku mamy komunikat tx
        # ustaw wartość True dla testnet tx
        # wykonaj w pętli dla wszystkich wyjść tx
            # jeśli wyjście ma ten sam adres, co nasz adres, wyjście zostało znalezione
                # znaleźliśmy nasze utxo, przypisz wartości do prev_tx, prev_index i tx
# utwórz TxIn
# oblicz kwotę wyjścia (poprzednia kwota minus opłata)
# utwórz nowe wyjście TxOut dla skryptu docelowego z kwotą wyjścia
# utwórz nową transakcję z jednym wejściem i z jednym wyjściem
# podpisz jedyne wejście transakcji
# serializuj i zamień na hex, aby zobaczyć, jak to wygląda
# wyślij tę podpisaną transakcję do sieci
# poczekaj sekundę, aż transakcja dotrze, używając time.sleep(1)
# potem zapytaj drugi węzeł o tę transakcję
# utwórz GetDataMessage
# zapytaj o naszą transakcję, dodając ją do komunikatu
# wyślij komunikat
# teraz poczekaj na odpowiedź Tx
# jeśli otrzymana tx ma ten sam identyfikator, co nasza tx, skończyliśmy!
